In [1]:
# ipcluster start -n 8 --engines=MPIEngineSetLauncher
import ipyparallel as ipp
client = ipp.Client(profile="default")
%autopx

%autopx enabled


/home/diego/Myopt/Miniconda/miniconda3/envs/UIBCDFLab_dev/lib/python3.6/site-packages/ipyparallel/client/client.py:459: RuntimeWarning: 
            Controller appears to be listening on localhost, but not on this machine.
            If this is true, you should specify Client(...,sshserver='you@nauta')
            or instruct your controller to listen on an external IP.
  RuntimeWarning)


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [3]:
import sys
sys.path.append('/home/diego/Projects/Pynterpred_devel/')
import time
import pynterpred as pnt
import numpy as np
from matplotlib import pyplot as plt

In [4]:
from mpi4py import MPI
comm = MPI.COMM_WORLD

In [5]:
Barnase     = '/home/diego/Projects/Pynterpred_devel/pynterpred/testsystems/Barnase-Barstar/pdbs/Barnase.pdb'
Barstar     = '/home/diego/Projects/Pynterpred_devel/pynterpred/testsystems/Barnase-Barstar/pdbs/Barstar.pdb'
force_field = 'amber14-all.xml'
pH          = 7.0

In [6]:
docking = pnt.predict(Barnase, Barstar, force_field, pH, delta_x = 0.8, nside = 3,
                      mpi_comm = comm, verbose = True)

[stdout:0] 
Setting up the mechanical molecular context... 4.78 secs
Setting up the evaluation region... 4.95 secs
Evaluation of 39960 different relative orientations started...
In mpi rank 0 centers from 0 to 46 in 30.71 seconds ( 1.5  its/sec)
Time to collect potential energies: 0.001814
31.41 secs
Done
Complex at infinite distance with Potential Energy: 7374.3515625 kJ/mol
Best relative orientation with Potential Energy: 5532.796875 kJ/mol
[stdout:1] In mpi rank 1 centers from 46 to 91 in 30.35 seconds ( 1.48  its/sec)
[stdout:2] In mpi rank 2 centers from 94 to 140 in 30.68 seconds ( 1.5  its/sec)
[stdout:3] In mpi rank 3 centers from 143 to 189 in 30.74 seconds ( 1.5  its/sec)
[stdout:4] In mpi rank 4 centers from 186 to 231 in 30.03 seconds ( 1.5  its/sec)
[stdout:5] In mpi rank 5 centers from 240 to 287 in 31.03 seconds ( 1.51  its/sec)
[stdout:6] In mpi rank 6 centers from 282 to 327 in 30.19 seconds ( 1.49  its/sec)
[stdout:7] In mpi rank 7 centers from 324 to 369 in 30.33 sec

In [ ]:
if comm.Get_rank() == 0:
    xx1, pes1 = docking.get_potential_energy_1D_landscape()

In [ ]:
if comm.Get_rank() == 0:
    print(docking.PotentialEnergyNetwork)

In [ ]:
docking = pnt.pynterpred(Barnase, Barstar, force_field, pH, delta_x = 0.5, nside = 5,
                         mpi_comm = comm, verbose = True)

xx1, pes1 = docking.get_potential_energy_1D_landscape()
del(docking)

In [ ]:
docking = pnt.pynterpred(Barnase, Barnase, force_field, pH, delta_x = 0.5, nside = 5, mpi_comm = comm)
xx2, pes2 = docking.get_potential_energy_1D_landscape()
del(docking)

In [ ]:
docking = pnt.pynterpred(Barstar, Barstar, force_field, pH, delta_x = 0.5, nside = 5, mpi_comm = comm)
xx3, pes3 = docking.get_potential_energy_1D_landscape()
del(docking)

In [ ]:
plt.scatter(xx1,pes1,0.1,label="Bb")
plt.scatter(xx2,pes2,0.1,label="BB")
plt.scatter(xx3,pes3,0.1,label="bb")
plt.legend()

plt.xlim([-10000,10000])
plt.ylim([-2000,4000])
